## At first we want to import all the modules which we want to in next 
as like ``import numpy as np, 
import pandas as pd, 
import seaborn as sb``

In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

### In next step we want to retrieve all data from data file like csv, exle, json etc 

In [4]:
zomato_data_set = pd.read_csv('../../asset/zomato/zomato.csv')

### Showing some data set what is it look like and understanding about data set 

In [6]:
zomato_data_set.head(1)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari


In [7]:
zomato_data_set.tail(1)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
51716,https://www.zomato.com/bangalore/the-nest-the-...,"ITPL Main Road, KIADB Export Promotion Industr...",The Nest - The Den Bengaluru,No,No,3.4 /5,13,+91 8071117272,"ITPL Main Road, Whitefield","Bar, Casual Dining",NaN,"Finger Food, North Indian, Continental","1,500","[('Rated 5.0', 'RATED\n Great ambience , look...",[],Pubs and bars,Whitefield


### For recommendations to user in this part we don't want to all data from here some unnecessary data we should drop from our data base 
like ``['url','dish_liked','phone']``

In [9]:
fresh_zomato_data = zomato_data_set.drop(['url','dish_liked','phone'], axis=1)

### now we get a fresh data set where have no unnecessary data and showings some data from new sata set

In [11]:
fresh_zomato_data.head(1)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari


### now check the data set here have any duplicate or not 

In [13]:
total_of_duplicate =  fresh_zomato_data.duplicated().sum()
print(total_of_duplicate)

43


### now we want to remove duplicate 

In [15]:
fresh_zomato_data.drop_duplicates(inplace=True)

#### Why use ``inplace=True`` if inplace=False it return new df if is True is return on original df

#### Now we want to count how many null value have in our data base

In [18]:
fresh_zomato_data.isnull()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51712,False,False,False,False,False,False,False,False,False,False,False,False,False,False
51713,False,False,False,False,True,False,False,False,False,False,False,False,False,False
51714,False,False,False,False,True,False,False,False,False,False,False,False,False,False
51715,False,False,False,False,False,False,False,False,False,False,False,False,False,False


##### if you just write ``fresh_zomato_data.isnull()`` this it provides full table with True and False if you want to see the total null in every Cl ans Row by then you must ``.sum()`` this full True False value by formate 0 and 1 

In [20]:
fresh_zomato_data.isnull().sum()

address                           0
name                              0
online_order                      0
book_table                        0
rate                           7767
votes                             0
location                         21
rest_type                       227
cuisines                         45
approx_cost(for two people)     345
reviews_list                      0
menu_item                         0
listed_in(type)                   0
listed_in(city)                   0
dtype: int64

#### Now you must remove NULL value or fill null field but in this case for number its ok but location and some important data which must provide by user in this case we must remove thos data row if any rows have NULL you must remove this 

In [22]:
fresh_zomato_data.dropna(how='any', inplace=True)

In [23]:
fresh_zomato_data.isnull().sum()

address                        0
name                           0
online_order                   0
book_table                     0
rate                           0
votes                          0
location                       0
rest_type                      0
cuisines                       0
approx_cost(for two people)    0
reviews_list                   0
menu_item                      0
listed_in(type)                0
listed_in(city)                0
dtype: int64

##### Now here have no NULL row field 

### We already completed some step of data operation like
- Deleting Unnecessary Columns
- Removing the Duplicates
- Remove the NaN values from the dataset
#### Now we want to ``Changing the column names`` with flexibile and understandable col name like `` approx_cost(for two people) `` to `` cost ``

In [26]:
fresh_zomato_data = fresh_zomato_data.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})

In [27]:
fresh_zomato_data.tail(1)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
51716,"ITPL Main Road, KIADB Export Promotion Industr...",The Nest - The Den Bengaluru,No,No,3.4 /5,13,"ITPL Main Road, Whitefield","Bar, Casual Dining","Finger Food, North Indian, Continental","1,500","[('Rated 5.0', 'RATED\n Great ambience , look...",[],Pubs and bars,Whitefield
